# SFScores - Predicting the Results of Health Inspections of Restaurants

In this notebook, we will benchmark several of getML's feature learning algorithms against featuretools using a dataset of eateries in San Francisco.

Summary:

- Prediction type: __Regression model__
- Domain: __Health__
- Prediction target: __Sales__ 
- Population size: __12887__

_Author: Dr. Patrick Urbanke_

# Background

This notebook is based on the San Francisco Dept. of Public Health's database of eateries in San Francisco. These eateries are regularly inspected. The inspections often result in a score.

The challenge is to predict the score resulting from an inspection.

The dataset has been downloaded from the [CTU Prague relational learning repository](https://relational.fit.cvut.cz/dataset/SFScores) (Motl and Schulte, 2015).

We will benchmark [getML](https://www.getml.com) 's feature learning algorithms against [featuretools](https://www.featuretools.com), an open-source implementation of the propositionalization algorithm, similar to getML's FastProp.

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

Let's get started with the analysis and set up your session:

In [1]:
import copy
import os
from urllib import request

import numpy as np
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline  

import featuretools
import getml

getml.engine.set_project('sfscores')



Loading pipelines...
[========================================] 100%


Connected to project 'sfscores'


## 1. Loading data

### 1.1 Download from source

We begin by downloading the data:

In [2]:
conn = getml.database.connect_mariadb(
    host="relational.fit.cvut.cz",
    dbname="SFScores",
    port=3306,
    user="guest",
    password="relational"
)

conn

Connection(conn_id='default',
           dbname='SFScores',
           dialect='mysql',
           host='relational.fit.cvut.cz',
           port=3306)

In [3]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if not getml.data.exists(name):
        data_frame = getml.data.DataFrame.from_db(
            name=name,
            table_name=name,
            conn=conn
        )
        data_frame.save()
    else:
        data_frame = getml.data.load_data_frame(name)
    return data_frame

In [4]:
businesses = load_if_needed("businesses")
inspections = load_if_needed("inspections")
violations = load_if_needed("violations")

In [5]:
businesses

name,business_id,latitude,longitude,phone_number,business_certificate,name,address,city,postal_code,tax_code,application_date,owner_name,owner_address,owner_city,owner_state,owner_zip
role,unused_float,unused_float,unused_float,unused_float,unused_float,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
0,10,37.7911,-122.404,nan,779059,Tiramisu Kitchen,033 Belden Pl,San Francisco,94104,H24,NULL,Tiramisu LLC,33 Belden St,San Francisco,CA,94104
1,24,37.7929,-122.403,nan,352312,OMNI S.F. Hotel - 2nd Floor Pant...,"500 California St, 2nd Floor",San Francisco,94104,H24,NULL,OMNI San Francisco Hotel Corp,"500 California St, 2nd Floor",San Francisco,CA,94104
2,31,37.8072,-122.419,nan,346882,Norman's Ice Cream and Freezes,2801 Leavenworth St,San Francisco,94133,H24,NULL,Norman Antiforda,2801 Leavenworth St,San Francisco,CA,94133
3,45,37.7471,-122.414,nan,340024,CHARLIE'S DELI CAFE,3202 FOLSOM St,S.F.,94110,H24,2001-10-10,"HARB, CHARLES AND KRISTIN",1150 SANCHEZ,S.F.,CA,94114
4,48,37.764,-122.466,nan,318022,ART'S CAFE,747 IRVING St,SAN FRANCISCO,94122,H24,NULL,YOON HAE RYONG,1567 FUNSTON AVE,SAN FRANCISCO,CA,94122
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6353,89335,nan,nan,nan,1057025,Breaking Bad Sandwiches,154 McAllister St,NULL,94102,H25,2016-09-23,"JPMD, LLC",662 Bellhurst Lane,Castro Valley,CA,94102
6354,89336,nan,nan,nan,1057746,Miller's Rest,1085 Sutter St,NULL,94109,H26,2016-09-23,"Miller's Rest, LLC",2906 Bush Street,San Francisco,CA,94109
6355,89393,nan,nan,nan,1042408,Panuchos,620 Broadway St,NULL,94133,H24,2016-09-28,"Los Aluxes, LLC","1032 Irving Street, #421",San Francisco,CA,94122


In [6]:
inspections

name,business_id,score,date,type
role,unused_float,unused_float,unused_string,unused_string
0,10,92,2014-01-14,Routine - Unscheduled
1,10,nan,2014-01-24,Reinspection/Followup
2,10,94,2014-07-29,Routine - Unscheduled
3,10,nan,2014-08-07,Reinspection/Followup
4,10,82,2016-05-03,Routine - Unscheduled
,...,...,...,...
23759,89199,100,2016-09-12,Routine - Unscheduled
23760,89200,100,2016-09-12,Routine - Unscheduled
23761,89201,nan,2016-09-12,New Ownership


In [7]:
violations

name,business_id,date,violation_type_id,risk_category,description
role,unused_float,unused_string,unused_string,unused_string,unused_string
0,10,2014-07-29,103129,Moderate Risk,Insufficient hot water or runnin...
1,10,2014-07-29,103144,Low Risk,Unapproved or unmaintained equip...
2,10,2014-01-14,103119,Moderate Risk,Inadequate and inaccessible hand...
3,10,2014-01-14,103145,Low Risk,Improper storage of equipment ut...
4,10,2014-01-14,103154,Low Risk,Unclean or degraded floors walls...
,...,...,...,...,...
36045,88878,2016-08-19,103144,Low Risk,Unapproved or unmaintained equip...
36046,88878,2016-08-19,103124,Moderate Risk,Inadequately cleaned or sanitize...
36047,89072,2016-09-22,103120,Moderate Risk,Moderate risk food holding tempe...


### 1.2 Prepare data for getML

getML requires that we define *roles* for each of the columns.

In [8]:
businesses.set_role("business_id", getml.data.roles.join_key)
businesses.set_role("name", getml.data.roles.text)
businesses.set_role(["postal_code", "tax_code", "owner_zip"], getml.data.roles.categorical)

businesses

name,business_id,postal_code,tax_code,owner_zip,name,latitude,longitude,phone_number,business_certificate,address,city,application_date,owner_name,owner_address,owner_city,owner_state
role,join_key,categorical,categorical,categorical,text,unused_float,unused_float,unused_float,unused_float,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
0,10,94104,H24,94104,Tiramisu Kitchen,37.7911,-122.404,nan,779059,033 Belden Pl,San Francisco,NULL,Tiramisu LLC,33 Belden St,San Francisco,CA
1,24,94104,H24,94104,OMNI S.F. Hotel - 2nd Floor Pant...,37.7929,-122.403,nan,352312,"500 California St, 2nd Floor",San Francisco,NULL,OMNI San Francisco Hotel Corp,"500 California St, 2nd Floor",San Francisco,CA
2,31,94133,H24,94133,Norman's Ice Cream and Freezes,37.8072,-122.419,nan,346882,2801 Leavenworth St,San Francisco,NULL,Norman Antiforda,2801 Leavenworth St,San Francisco,CA
3,45,94110,H24,94114,CHARLIE'S DELI CAFE,37.7471,-122.414,nan,340024,3202 FOLSOM St,S.F.,2001-10-10,"HARB, CHARLES AND KRISTIN",1150 SANCHEZ,S.F.,CA
4,48,94122,H24,94122,ART'S CAFE,37.764,-122.466,nan,318022,747 IRVING St,SAN FRANCISCO,NULL,YOON HAE RYONG,1567 FUNSTON AVE,SAN FRANCISCO,CA
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6353,89335,94102,H25,94102,Breaking Bad Sandwiches,nan,nan,nan,1057025,154 McAllister St,NULL,2016-09-23,"JPMD, LLC",662 Bellhurst Lane,Castro Valley,CA
6354,89336,94109,H26,94109,Miller's Rest,nan,nan,nan,1057746,1085 Sutter St,NULL,2016-09-23,"Miller's Rest, LLC",2906 Bush Street,San Francisco,CA
6355,89393,94133,H24,94122,Panuchos,nan,nan,nan,1042408,620 Broadway St,NULL,2016-09-28,"Los Aluxes, LLC","1032 Irving Street, #421",San Francisco,CA


In [9]:
inspections = inspections[~inspections.score.is_nan()].to_df("inspections")

inspections.set_role("business_id", getml.data.roles.join_key)
inspections.set_role("score", getml.data.roles.target)
inspections.set_role("date", getml.data.roles.time_stamp)

inspections

name,date,business_id,score,type
role,time_stamp,join_key,target,unused_string
unit,"time stamp, comparison only",,,
0,2014-01-14,10,92,Routine - Unscheduled
1,2014-07-29,10,94,Routine - Unscheduled
2,2016-05-03,10,82,Routine - Unscheduled
3,2013-11-18,24,100,Routine - Unscheduled
4,2014-06-12,24,96,Routine - Unscheduled
,...,...,...,...
12882,2016-09-22,89072,90,Routine - Unscheduled
12883,2016-09-12,89198,100,Routine - Unscheduled


In [10]:
violations.set_role("business_id", getml.data.roles.join_key)
violations.set_role("date", getml.data.roles.time_stamp)
violations.set_role(["violation_type_id", "risk_category"], getml.data.roles.categorical)
violations.set_role("description", getml.data.roles.text)

violations

name,date,business_id,violation_type_id,risk_category,description
role,time_stamp,join_key,categorical,categorical,text
unit,"time stamp, comparison only",,,,
0,2014-07-29,10,103129,Moderate Risk,Insufficient hot water or runnin...
1,2014-07-29,10,103144,Low Risk,Unapproved or unmaintained equip...
2,2014-01-14,10,103119,Moderate Risk,Inadequate and inaccessible hand...
3,2014-01-14,10,103145,Low Risk,Improper storage of equipment ut...
4,2014-01-14,10,103154,Low Risk,Unclean or degraded floors walls...
,...,...,...,...,...
36045,2016-08-19,88878,103144,Low Risk,Unapproved or unmaintained equip...
36046,2016-08-19,88878,103124,Moderate Risk,Inadequately cleaned or sanitize...


## 2. Predictive modeling

We loaded the data and defined the roles and units. Next, we create a getML pipeline for relational learning.

In [11]:
split = getml.data.split.random(train=0.8, test=0.2)

### 2.1 Define relational model

In [12]:
star_schema = getml.data.StarSchema(population=inspections, alias="population", split=split)

star_schema.join(
    businesses,
    on="business_id",
    relationship=getml.data.relationship.many_to_one,
)

star_schema.join(
    violations,
    on="business_id",
    time_stamps="date",
    horizon=getml.data.time.days(1),
)

star_schema.join(
    inspections,
    on="business_id",
    time_stamps="date",
    lagged_targets=True,
    horizon=getml.data.time.days(1),
)

star_schema

data model

  population:
    columns:
    - business_id: join_key
    - score: target
    - date: time_stamp
    - type: unused_string

    joins:
    - right: 'businesses'
      on: (population.business_id, businesses.business_id)
      relationship: 'many-to-one'
      lagged_targets: False
    - right: 'violations'
      on: (population.business_id, violations.business_id)
      time_stamps: (population.date, violations.date)
      relationship: 'many-to-many'
      horizon: 86400.0
      lagged_targets: False
    - right: 'inspections'
      on: (population.business_id, inspections.business_id)
      time_stamps: (population.date, inspections.date)
      relationship: 'many-to-many'
      horizon: 86400.0
      lagged_targets: True

  businesses:
    columns:
    - postal_code: categorical
    - tax_code: categorical
    - owner_zip: categorical
    - business_id: join_key
    - name: text
    - ...

  violations:
    columns:
    - violation_type_id: categorical
    - risk_category: categorical
    - business_id: join_key
    - description: text
    - date: time_stamp

  inspections:
    columns:
    - business_id: join_key
    - score: target
    - date: time_stamp
    - type: unused_string


container

  population
      subset   name           rows   type
  0   test     inspections    2492   View
  1   train    inspections   10395   View

  peripheral
      name           rows   type     
  0   businesses     6358   DataFrame
  1   violations    36050   DataFrame
  2   inspections   12887   DataFrame

### 2.2 getML pipeline

<!-- #### 2.1.1  -->
__Set-up the feature learner & predictor__

We use the relboost algorithms for this problem. Because of the large number of keywords, we regularize the model a bit by requiring a minimum support for the keywords (`min_num_samples`).

In [13]:
mapping = getml.preprocessors.Mapping()

fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
    num_threads=1,
)

relboost = getml.feature_learning.Relboost(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
)

relmt = getml.feature_learning.RelMT(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

__Build the pipeline__

In [14]:
pipe1 = getml.pipeline.Pipeline(
    tags=['fast_prop'],
    data_model=star_schema.data_model,
    preprocessors=[mapping],
    feature_learners=[fast_prop],
    predictors=[predictor]
)

pipe1

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=['businesses', 'inspections', 'violations'],
         predictors=['XGBoostRegressor'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop'])

### 2.3 Model training

In [15]:
pipe1.check(star_schema.train)

Checking data model...


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

Checking...
[========================================] 100%


INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and VIOLATIONS__STAGING_TABLE_3 over 'business_id' and 'business_id', there are no corresponding entries for 5.685426% of entries in 'business_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.


In [16]:
pipe1.fit(star_schema.train)

Checking data model...


Staging...
[========================================] 100%


INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and VIOLATIONS__STAGING_TABLE_3 over 'business_id' and 'business_id', there are no corresponding entries for 5.685426% of entries in 'business_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

Indexing text fields...
[========================================] 100%

FastProp: Trying 336 features...
[========================================] 100%

FastProp: Building features...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%


Trained pipeline.
Time taken: 0h:0m:12.649578



Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=['businesses', 'inspections', 'violations'],
         predictors=['XGBoostRegressor'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop', 'container-EOXloA'])

url: http://localhost:1709/#/getpipeline/sfscores/ZTWoNv/0/

### 2.4 Model evaluation

In [17]:
pipe1.score(star_schema.test)



Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

FastProp: Building features...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-08-24 12:37:04,train,score,4.904,6.537,0.3601
1,2021-08-24 12:37:05,test,score,5.3698,7.1009,0.2793


### 2.5 featuretools

In [18]:
population_train_pd = star_schema.train.population.to_pandas()
population_test_pd = star_schema.test.population.to_pandas()

In [19]:
inspections_pd = inspections.drop(inspections.roles.unused).to_pandas()
violations_pd = violations.drop(violations.roles.unused).to_pandas()
businesses_pd = businesses.drop(businesses.roles.unused).to_pandas()

In [20]:
population_train_pd["id"] = population_train_pd.index

population_train_pd = population_train_pd.merge(
    businesses_pd,
    on="business_id"
)

population_train_pd

,date,business_id,score,id,postal_code,tax_code,owner_zip,name
0,2014-01-14,10,92.0,0,94104,H24,94104,Tiramisu Kitchen
1,2014-07-29,10,94.0,1,94104,H24,94104,Tiramisu Kitchen
2,2016-05-03,10,82.0,2,94104,H24,94104,Tiramisu Kitchen
3,2014-06-12,24,96.0,3,94104,H24,94104,OMNI S.F. Hotel - 2nd Floor Pantry
4,2014-11-24,24,96.0,4,94104,H24,94104,OMNI S.F. Hotel - 2nd Floor Pantry
...,...,...,...,...,...,...,...,...
10390,2016-08-19,88878,94.0,10390,94102,H24,94566,Jamba Juice
10391,2016-09-22,89072,90.0,10391,94109,H91,94109,Epicurean at Sacred Heart Catholic Prep School
10392,2016-09-12,89198,100.0,10392,94107,H36,29615,"AT&T Park - Beer Cart/View Level, Sec. 333"
10393,2016-09-12,89199,100.0,10393,94107,H36,29615,"AT&T Park - Beer Cart/Lower CF, Sec. 140"


In [21]:
population_test_pd["id"] = population_test_pd.index

population_test_pd = population_test_pd.merge(
    businesses_pd,
    on="business_id"
)

population_test_pd

,date,business_id,score,id,postal_code,tax_code,owner_zip,name
0,2013-11-18,24,100.0,0,94104,H24,94104,OMNI S.F. Hotel - 2nd Floor Pantry
1,2016-03-11,24,96.0,1,94104,H24,94104,OMNI S.F. Hotel - 2nd Floor Pantry
2,2013-12-09,45,94.0,2,94110,H24,94114,CHARLIE'S DELI CAFE
3,2014-07-25,58,78.0,3,94111,H24,94111,Oasis Grill
4,2014-05-19,66,91.0,4,94122,H24,94122,STARBUCKS
...,...,...,...,...,...,...,...,...
2487,2016-06-07,87802,91.0,2487,94110,H25,94110,Bernal Heights Pizzeria
2488,2016-08-30,88082,84.0,2488,94133,H24,94133,Chongqing Xiaomian
2489,2016-08-17,88447,96.0,2489,NULL,H91,94107,Fare Resources
2490,2016-08-15,88702,96.0,2490,94118,H25,94118,Dancing Bull


In [22]:
def prepare_peripheral(violations_pd, train_or_test):
    """
    Helper function that imitates the behavior of 
    the data model defined above.
    """
    violations_new = violations_pd.merge(
        train_or_test[["id", "business_id", "date"]],
        on="business_id"
    )

    violations_new = violations_new[
        violations_new["date_x"] < violations_new["date_y"]
    ]

    del violations_new["date_y"]
    del violations_new["business_id"]

    return violations_new.rename(columns={"date_x": "date"})

In [23]:
violations_train_pd = prepare_peripheral(violations_pd, population_train_pd)
violations_test_pd = prepare_peripheral(violations_pd, population_test_pd)
violations_train_pd

,date,violation_type_id,risk_category,description,id
2,2014-07-29,103129,Moderate Risk,Insufficient hot water or running water,2
5,2014-07-29,103144,Low Risk,Unapproved or unmaintained equipment or utensils,2
7,2014-01-14,103119,Moderate Risk,Inadequate and inaccessible handwashing facili...,1
8,2014-01-14,103119,Moderate Risk,Inadequate and inaccessible handwashing facili...,2
10,2014-01-14,103145,Low Risk,Improper storage of equipment utensils or linens,1
...,...,...,...,...,...
89220,2016-02-16,103119,Moderate Risk,Inadequate and inaccessible handwashing facili...,10290
89256,2016-04-04,103131,Moderate Risk,Moderate risk vermin infestation,10308
89336,2016-04-11,103154,Low Risk,Unclean or degraded floors walls or ceilings,10331
89338,2016-04-11,103148,Low Risk,No thermometers or uncalibrated thermometers,10331


In [24]:
inspections_train_pd = prepare_peripheral(inspections_pd, population_train_pd)
inspections_test_pd = prepare_peripheral(inspections_pd, population_test_pd)
inspections_train_pd

,date,score,id
1,2014-01-14,92.0,1
2,2014-01-14,92.0,2
5,2014-07-29,94.0,2
9,2013-11-18,100.0,3
10,2013-11-18,100.0,4
...,...,...,...
32628,2016-02-16,92.0,10290
32648,2016-04-04,96.0,10308
32673,2016-04-11,94.0,10331
32707,2016-05-23,100.0,10360


In [25]:
del population_train_pd["business_id"]
del population_test_pd["business_id"]

In [26]:
population_train_pd

,date,score,id,postal_code,tax_code,owner_zip,name
0,2014-01-14,92.0,0,94104,H24,94104,Tiramisu Kitchen
1,2014-07-29,94.0,1,94104,H24,94104,Tiramisu Kitchen
2,2016-05-03,82.0,2,94104,H24,94104,Tiramisu Kitchen
3,2014-06-12,96.0,3,94104,H24,94104,OMNI S.F. Hotel - 2nd Floor Pantry
4,2014-11-24,96.0,4,94104,H24,94104,OMNI S.F. Hotel - 2nd Floor Pantry
...,...,...,...,...,...,...,...
10390,2016-08-19,94.0,10390,94102,H24,94566,Jamba Juice
10391,2016-09-22,90.0,10391,94109,H91,94109,Epicurean at Sacred Heart Catholic Prep School
10392,2016-09-12,100.0,10392,94107,H36,29615,"AT&T Park - Beer Cart/View Level, Sec. 333"
10393,2016-09-12,100.0,10393,94107,H36,29615,"AT&T Park - Beer Cart/Lower CF, Sec. 140"


In [27]:
entities_train = {
    "population" : (population_train_pd, "id"),
    "inspections" : (inspections_train_pd, "index"),
    "violations" : (violations_train_pd, "index")
}

In [28]:
entities_test = {
    "population" : (population_test_pd, "id"),
    "inspections" : (inspections_test_pd, "index"),
    "violations" : (violations_test_pd, "index")
}

In [29]:
relationships = [
    ("population", "id", "inspections", "id"),
    ("population", "id", "violations", "id")
]

In [30]:
featuretools_train_pd = featuretools.dfs(
    entities=entities_train,
    relationships=relationships,
    target_entity="population")[0]

/home/patrick/.local/lib/python3.9/site-packages/featuretools/entityset/entity.py:462: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn("index {} not found in dataframe, creating new "


In [31]:
featuretools_test_pd = featuretools.dfs(
    entities=entities_test,
    relationships=relationships,
    target_entity="population")[0]

In [32]:
featuretools_train = getml.data.DataFrame.from_pandas(featuretools_train_pd, "featuretools_train")
featuretools_test = getml.data.DataFrame.from_pandas(featuretools_test_pd, "featuretools_test")

In [33]:
featuretools_train.set_role("name", getml.data.roles.text)
featuretools_train.set_role("score", getml.data.roles.target)
featuretools_train.set_role(featuretools_train.roles.unused_float, getml.data.roles.numerical)
featuretools_train.set_role(featuretools_train.roles.unused_string, getml.data.roles.categorical)

featuretools_train

name,score,postal_code,tax_code,owner_zip,MODE(violations.description),MODE(violations.risk_category),MODE(violations.violation_type_id),COUNT(inspections),MAX(inspections.score),MEAN(inspections.score),MIN(inspections.score),SKEW(inspections.score),STD(inspections.score),SUM(inspections.score),COUNT(violations),NUM_UNIQUE(violations.description),NUM_UNIQUE(violations.risk_category),NUM_UNIQUE(violations.violation_type_id),DAY(date),MONTH(date),WEEKDAY(date),YEAR(date),MODE(inspections.DAY(date)),MODE(inspections.MONTH(date)),MODE(inspections.WEEKDAY(date)),MODE(inspections.YEAR(date)),NUM_UNIQUE(inspections.DAY(date)),NUM_UNIQUE(inspections.MONTH(date)),NUM_UNIQUE(inspections.WEEKDAY(date)),NUM_UNIQUE(inspections.YEAR(date)),MODE(violations.DAY(date)),MODE(violations.MONTH(date)),MODE(violations.WEEKDAY(date)),MODE(violations.YEAR(date)),NUM_UNIQUE(violations.DAY(date)),NUM_UNIQUE(violations.MONTH(date)),NUM_UNIQUE(violations.WEEKDAY(date)),NUM_UNIQUE(violations.YEAR(date)),name
role,target,categorical,categorical,categorical,categorical,categorical,categorical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,text
0,92,94104,H24,94104,NULL,NULL,NULL,0,nan,nan,nan,nan,nan,0,0,nan,nan,nan,14,1,1,2014,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Tiramisu Kitchen
1,94,94104,H24,94104,Improper storage of equipment ut...,Low Risk,103119,1,92,92,92,nan,nan,92,3,3,2,3,29,7,1,2014,14,1,1,2014,1,1,1,1,14,1,1,2014,1,1,1,1,Tiramisu Kitchen
2,82,94104,H24,94104,Improper storage of equipment ut...,Low Risk,103119,2,94,93,92,nan,1.4142,186,5,5,2,5,3,5,1,2016,14,1,1,2014,2,2,1,1,14,1,1,2014,2,2,1,1,Tiramisu Kitchen
3,96,94104,H24,94104,NULL,NULL,NULL,1,100,100,100,nan,nan,100,0,nan,nan,nan,12,6,3,2014,18,11,0,2013,1,1,1,1,nan,nan,nan,nan,nan,nan,nan,nan,OMNI S.F. Hotel - 2nd Floor Pant...
4,96,94104,H24,94104,Improper storage of equipment ut...,Low Risk,103145,2,100,98,96,nan,2.8284,196,2,2,1,2,24,11,0,2014,12,6,0,2013,2,2,2,2,12,6,3,2014,1,1,1,1,OMNI S.F. Hotel - 2nd Floor Pant...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10390,94,94102,H24,94566,NULL,NULL,NULL,0,nan,nan,nan,nan,nan,0,0,nan,nan,nan,19,8,4,2016,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Jamba Juice
10391,90,94109,H91,94109,NULL,NULL,NULL,0,nan,nan,nan,nan,nan,0,0,nan,nan,nan,22,9,3,2016,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Epicurean at Sacred Heart Cathol...
10392,100,94107,H36,29615,NULL,NULL,NULL,0,nan,nan,nan,nan,nan,0,0,nan,nan,nan,12,9,0,2016,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,AT&T Park - Beer Cart/View Level...


In [34]:
featuretools_test.set_role("name", getml.data.roles.text)
featuretools_test.set_role("score", getml.data.roles.target)
featuretools_test.set_role(featuretools_test.roles.unused_float, getml.data.roles.numerical)
featuretools_test.set_role(featuretools_test.roles.unused_string, getml.data.roles.categorical)

featuretools_test

name,score,postal_code,tax_code,owner_zip,MODE(violations.description),MODE(violations.risk_category),MODE(violations.violation_type_id),COUNT(inspections),MAX(inspections.score),MEAN(inspections.score),MIN(inspections.score),SKEW(inspections.score),STD(inspections.score),SUM(inspections.score),COUNT(violations),NUM_UNIQUE(violations.description),NUM_UNIQUE(violations.risk_category),NUM_UNIQUE(violations.violation_type_id),DAY(date),MONTH(date),WEEKDAY(date),YEAR(date),MODE(inspections.DAY(date)),MODE(inspections.MONTH(date)),MODE(inspections.WEEKDAY(date)),MODE(inspections.YEAR(date)),NUM_UNIQUE(inspections.DAY(date)),NUM_UNIQUE(inspections.MONTH(date)),NUM_UNIQUE(inspections.WEEKDAY(date)),NUM_UNIQUE(inspections.YEAR(date)),MODE(violations.DAY(date)),MODE(violations.MONTH(date)),MODE(violations.WEEKDAY(date)),MODE(violations.YEAR(date)),NUM_UNIQUE(violations.DAY(date)),NUM_UNIQUE(violations.MONTH(date)),NUM_UNIQUE(violations.WEEKDAY(date)),NUM_UNIQUE(violations.YEAR(date)),name
role,target,categorical,categorical,categorical,categorical,categorical,categorical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,text
0,100,94104,H24,94104,NULL,NULL,NULL,0,nan,nan,nan,nan,nan,0,0,nan,nan,nan,18,11,0,2013,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,OMNI S.F. Hotel - 2nd Floor Pant...
1,96,94104,H24,94104,Improper storage of equipment ut...,Low Risk,103119,3,100,97.3333,96,1.7321,2.3094,292,3,3,2,3,11,3,4,2016,12,11,0,2014,3,2,2,2,12,6,3,2014,2,2,2,1,OMNI S.F. Hotel - 2nd Floor Pant...
2,94,94110,H24,94114,NULL,NULL,NULL,0,nan,nan,nan,nan,nan,0,0,nan,nan,nan,9,12,0,2013,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,CHARLIE'S DELI CAFE
3,78,94111,H24,94111,NULL,NULL,NULL,0,nan,nan,nan,nan,nan,0,0,nan,nan,nan,25,7,4,2014,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Oasis Grill
4,91,94122,H24,94122,Wiping cloths not clean or prope...,Low Risk,103149,1,98,98,98,nan,nan,98,1,1,1,1,19,5,0,2014,10,2,0,2014,1,1,1,1,10,2,0,2014,1,1,1,1,STARBUCKS
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2487,91,94110,H25,94110,NULL,NULL,NULL,0,nan,nan,nan,nan,nan,0,0,nan,nan,nan,7,6,1,2016,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Bernal Heights Pizzeria
2488,84,94133,H24,94133,NULL,NULL,NULL,0,nan,nan,nan,nan,nan,0,0,nan,nan,nan,30,8,1,2016,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Chongqing Xiaomian
2489,96,NULL,H91,94107,NULL,NULL,NULL,0,nan,nan,nan,nan,nan,0,0,nan,nan,nan,17,8,2,2016,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Fare Resources


We train an untuned XGBoostRegressor on top of featuretools' features, just like we have done for getML's features.

Since some of featuretools features are categorical, we allow the pipeline to include these features as well. Other features contain NaN values, which is why we also apply getML's Imputation preprocessor.

In [35]:
imputation = getml.preprocessors.Imputation()

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe2 = getml.pipeline.Pipeline(
    tags=['featuretools'],
    preprocessors=[imputation],
    predictors=[predictor],
    include_categorical=True,
)

pipe2

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=['Imputation'],
         share_selected_features=0.5,
         tags=['featuretools'])

In [36]:
pipe2.fit(featuretools_train)

Checking data model...


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

Checking...
[========================================] 100%


OK.


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%


Trained pipeline.
Time taken: 0h:0m:1.963136



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=['Imputation'],
         share_selected_features=0.5,
         tags=['featuretools'])

url: http://localhost:1709/#/getpipeline/sfscores/FtzMya/0/

In [37]:
pipe2.score(featuretools_test)



Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-08-24 12:37:47,featuretools_train,score,5.1038,6.7785,0.3111
1,2021-08-24 12:37:47,featuretools_test,score,5.4529,7.1896,0.2618


### 2.6 Productionization

It is possible to productionize the pipeline by transpiling the features into production-ready SQL code. Please also refer to getML's `sqlite3` module.

In [38]:
# Creates a folder named sfscores_pipeline containing
# the SQL code.
pipe1.features.to_sql().save("sfscores_pipeline")

### 2.7 Discussion

For a more convenient overview, we summarize our results into a table.

Name                 | R-squared | RMSE | MAE
-------------------- | --------- | ---- | ----
getML: FastProp      |    27.93% | 7.10 | 5.37
featuretools         |    26.18% | 7.18 | 5.45

As we can see, getML's FastProp outperforms featuretools according to all three measures.

## 3. Conclusion

We have benchmarked getML against featuretools on dataset related to health inspections of eateries in San Francisco. We have found that getML outperforms featuretools.

## References

Motl, Jan, and Oliver Schulte. "The CTU prague relational learning repository." arXiv preprint arXiv:1511.03086 (2015).

# Next Steps

This tutorial benchmarked getML against academic state-of-the-art algorithms from relational learning literature and getML's qualities with respect to categorical data.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.